In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from lifetimes import utils as lifetimes_utils

# --- Configuration for Simulation ---
N_CUSTOMERS = 4000  # Target number of unique high-value patrons
START_DATE = datetime(2024, 1, 1)
END_DATE = datetime(2025, 12, 31)
N_DAYS = (END_DATE - START_DATE).days

# --- PHASE A: Simulate Raw Transaction Log (Step A1 - A3) ---
print("1. Simulating Raw Transaction Log...")

# 1. Generate Customer IDs
customer_ids = np.array([f'MGM_{i:04d}' for i in range(1, N_CUSTOMERS + 1)])

# 2. Simulate Transaction Frequency and Volume
# We aim for ~100,000 transactions total (avg 25 per customer)
N_TRANSACTIONS = 100000

# Randomly select customers, giving a slight edge to some to simulate 'High-Value' concentration
simulated_customer_ids = np.random.choice(customer_ids, size=N_TRANSACTIONS, p=None)

# 3. Simulate Transaction Dates
# Randomly select a day within the 24-month period
transaction_days = np.random.randint(0, N_DAYS, size=N_TRANSACTIONS)
transaction_dates = [START_DATE + timedelta(days=int(d)) for d in transaction_days]

# 4. Simulate Net Revenue (Skewed to simulate high-value patrons)
# Use a Gamma distribution to simulate high skew (many small values, few large ones)
# Mean transaction value is set high to reflect 'High-Value Patrons' (e.g., $400)
revenue_shape = 2.0
revenue_scale = 200
net_revenue = np.random.gamma(revenue_shape, revenue_scale, N_TRANSACTIONS)
# Ensure all revenue is positive
net_revenue = np.maximum(net_revenue, 5.0)

# 5. Simulate Contextual Features
channels = np.random.choice(['Gaming', 'Hotel', 'Dining', 'Retail'], size=N_TRANSACTIONS, p=[0.4, 0.3, 0.2, 0.1])
promotion_indicator = np.random.randint(0, 2, size=N_TRANSACTIONS) # 0 or 1

# Create the final DataFrame
raw_transactions_df = pd.DataFrame({
    'Customer_ID': simulated_customer_ids,
    'Transaction_Date': transaction_dates,
    'Net_Revenue': net_revenue.round(2),
    'Channel': channels,
    'Promotion_Indicator': promotion_indicator
})

# Sort by customer and date for realism
raw_transactions_df.sort_values(by=['Customer_ID', 'Transaction_Date'], inplace=True)
raw_transactions_df.reset_index(drop=True, inplace=True)

print(f"   -> Raw Transactions simulated: {len(raw_transactions_df):,} rows.")
print(raw_transactions_df.head())
print("-" * 50)


# --- PHASE B: Feature Engineering (Steps B4 & B5) ---
print("2. Generating RFM Summary Table (Feature Engineering)...")

# 1. Define Training and Prediction Periods
# T_CALIBRATION is the split date: 2024-12-31
T_CALIBRATION = datetime(2024, 12, 31)

# 2. Calculate RFM Features (Training Set) - Step B4
# Use the 'lifetimes' utility to transform transaction data into the required RFM summary
# T (Age) and Recency (t_x) are calculated relative to the T_CALIBRATION date.
rfm_df_raw = lifetimes_utils.summary_data_from_transaction_data(
    raw_transactions_df,
    customer_id_col='Customer_ID',
    datetime_col='Transaction_Date',
    monetary_value_col='Net_Revenue',
    # Calculate RFM only based on data *before* the split date
    observation_period_end=T_CALIBRATION,
    freq='D' # Calculate Recency/T in days
)

# Rename columns to the standard BG/NBD notation for clarity
rfm_df = rfm_df_raw.rename(columns={
    'frequency': 'x',         # Frequency
    'recency': 't_x',         # Recency (age of last purchase)
    'T': 'T',                 # Customer Age
    'monetary_value': 'm'     # Average Monetary Value (Monetary)
})

# 3. Define Target Variable (Actual CLV) - Step B5
# Identify transactions that occurred *after* the calibration date (the 12-month prediction period)
prediction_period_txns = raw_transactions_df[raw_transactions_df['Transaction_Date'] > T_CALIBRATION].copy()

# Sum the revenue in the prediction period for each customer
target_clv_df = prediction_period_txns.groupby('Customer_ID')['Net_Revenue'].sum().reset_index()
target_clv_df.rename(columns={'Net_Revenue': 'Actual_12m_CLV'}, inplace=True)

# Merge the target CLV back into the RFM summary table
rfm_df = rfm_df.merge(target_clv_df, on='Customer_ID', how='left').fillna(0)


# --- FINAL DATASET READY FOR MODEL TRAINING ---
print(f"   -> RFM Summary Table generated: {len(rfm_df):,} customers.")
print("\nFinal Dataset (Input for CLV Models):")
# Filter out customers with 0 frequency (no repeat purchase, cannot train Gamma-Gamma)
# Also filter out customers who never transacted in the training period (T=0)
rfm_df = rfm_df[(rfm_df['x'] > 0) & (rfm_df['T'] > 0)]

print(f"   -> Customers ready for modeling (x > 0 and T > 0): {len(rfm_df):,} customers.")
print(rfm_df.describe().T[['count', 'mean', 'std', 'min', 'max']])

# Save the RFM table to CSV for further use in Phase C
rfm_df.to_csv('clv_rfm_summary_data.csv', index=True)
print("\nRFM data saved to 'clv_rfm_summary_data.csv'")

1. Simulating Raw Transaction Log...
   -> Raw Transactions simulated: 100,000 rows.
  Customer_ID Transaction_Date  Net_Revenue Channel  Promotion_Indicator
0    MGM_0001       2024-01-03       321.80  Retail                    1
1    MGM_0001       2024-03-20        67.70  Gaming                    0
2    MGM_0001       2024-04-30       549.25  Dining                    0
3    MGM_0001       2024-04-30       634.63   Hotel                    0
4    MGM_0001       2024-05-13       301.48   Hotel                    0
--------------------------------------------------
2. Generating RFM Summary Table (Feature Engineering)...
   -> RFM Summary Table generated: 4,000 customers.

Final Dataset (Input for CLV Models):
   -> Customers ready for modeling (x > 0 and T > 0): 3,999 customers.
                 count         mean          std      min           max
x               3999.0    11.322581     3.471310    1.000     27.000000
t_x             3999.0   308.033508    40.134864   73.000    36